This notebook is going to be for processing and quality control of the hashtagged & comoplete data. Begin with package loading and a few functions definition. (should probably make a script i can read in of my functions at some point)

In [106]:
import os
import gc
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from solo import hashsolo
import scvi
import matplotlib.pyplot as plt


In [90]:
# Function which prints basic Quality Control metrics without any filtering/changing data
def summarize_adata(adata):
    # Calculate the total number of cells and genes
    total_cells = adata.n_obs
    total_genes = adata.n_vars
    
    # Calculate number of cells with fewer than 500 UMI counts
    cells_with_500_or_more = (adata.X.sum(axis=1) >= 500).sum()
    cells_with_less_than_500 = total_cells - cells_with_500_or_more
    print(f"Number of cells with fewer than 500 UMI counts: {cells_with_less_than_500}")

    # Calculate number of cells with fewer than 200 genes
    cells_with_200_or_more_genes = (adata.X > 0).sum(axis=1) >= 200
    cells_with_less_than_200_genes = total_cells - cells_with_200_or_more_genes.sum()
    print(f"Number of cells with fewer than 200 genes: {cells_with_less_than_200_genes}")

    # Calculate number of genes expressed in fewer than 3 cells
    genes_with_3_or_more_cells = (adata.X > 0).sum(axis=0) >= 3
    genes_with_less_than_3_cells = total_genes - genes_with_3_or_more_cells.sum()
    print(f"Number of genes expressed in fewer than 3 cells: {genes_with_less_than_3_cells}")
    

    # Print current status of the AnnData object
    print(f"Current Anndata has {adata.n_obs} cells and {adata.n_vars} genes, with a total amount of {adata.X.sum()} UMI counts")

def QC_filter_adata(adata):
    # Annotate the group of mitochondrial genes as 'mt'
    adata.var["mt"] = adata.var_names.str.startswith("mt-")

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

    # Visualize QC metrics before filtering
    print("Visualizations before filtering:")
    sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], jitter=0.4, multi_panel=True)
    sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", color="pct_counts_mt")

    # Print the number of cells with more than 5% mitochondrial genes
    cells_with_more_than_5_percent_mt = (adata.obs["pct_counts_mt"] > 5).sum()
    print(f"{cells_with_more_than_5_percent_mt} of {adata.n_obs} cells contain more than 5% mitochondrial genes. Will be filtered")

    # Filter cells with more than 5% mitochondrial genes
    adata = adata[adata.obs.pct_counts_mt < 5, :]
    sc.pp.filter_cells(adata, min_counts=500)
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)

    # Recalculate QC metrics for the filtered data
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

    # Visualize QC metrics after filtering
    print("Visualizations after filtering:")
    sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], jitter=0.4, multi_panel=True)
    sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", color="pct_counts_mt")

    return adata

# Function to generate a dictionary of lists of genes relevant to analysis which are present in the dataset
def find_genes(adata):
    #define some lists of genes of interest for the analysis.
    Genes_of_interest = []
    homeo_genes = ['Sall1', 'Cx3cr1', 'P2ry12', 'P2ry13', 'Olfml3', 'Tmem119', 'Cd68', 'Itgam', 'Cst3']
    DAM_genes = ['Apoe', 'B2m', 'Ctsb', 'Lpl', 'Cst7', 'Tyrobp', 'Trem2', 'Cd9', 'Itgax', 'Cd63', 'Fth1', 'Spp1', 'Axl', 'Mertk', 'Mdk', 'Ptn']
    Mapk_genes = ['Syk', 'Prkca', 'Mef2c', 'Elk4', 'Trp53', 'Mertk', 'Axl', 'Mapk14', 'Mapk1', 'Gadd45a',]
    Nfkb_genes = ['Nfkb1', 'Tlr4', 'Nfkbia', 'Cd40', 'Tlr4', 'Rela', 'Relb']
    Macroph_genes = ['Mrc1', 'Mrc2', 'Ccr2', 'Ly6c2', 'Lyz2', 'Vim', 'Ifi204', 'S100a10', 'Msrb1']
    # make a dictionary of the lists to iterate through
    gene_lists = {"Homeo": homeo_genes, "DAM": DAM_genes, "Mapk": Mapk_genes, "Nfkb": Nfkb_genes, "Macroph": Macroph_genes}
    found_genes = {key: [] for key in gene_lists.keys()}

    # loop through gene lists and genes, see if it is present in the passed adata, if so append it to a list+dictionary 
    # so we have a dicitonary of lists of only genes in the dataset, as well as a list of all genes from the different types
    for gene_list_name, gene_list in gene_lists.items():
        for gene in gene_list:
            if gene in adata.var.index:
                print(f" ✓ {gene} is in adata.var")
                Genes_of_interest.append(gene)
                found_genes[gene_list_name].append(gene)
            else:
                print(f"{gene} is not in adata.var")

    return Genes_of_interest, found_genes


In [ ]:
# # Read each sequencing run data into a dictionary, with a name for each as the key
# adatas = {
#     "D1" : sc.read_10x_mtx(
#         "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/collect_tube_1_batch_3_June_outs/filtered_feature_bc_matrix",
#         var_names="gene_symbols",
#         cache=False,
#         make_unique=True,
#         gex_only=False),
#     "D2" : sc.read_10x_mtx(
#         "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/Isol_Microglia_EFAD_TD_august_outs/filtered_feature_bc_matrix",
#         var_names="gene_symbols",
#         cache=False,
#         make_unique=True,
#         gex_only=False),
#     "D3" : sc.read_10x_mtx(
#         "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/Tube2_July_20_TD_outs/filtered_feature_bc_matrix",
#         var_names="gene_symbols",
#         cache=False,
#         make_unique=True,
#         gex_only=False),
#     "D4" : sc.read_10x_mtx(
#         "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/Tube3_july_20_TD_outs/filtered_feature_bc_matrix",
#         var_names="gene_symbols",
#         cache=False,
#         make_unique=True,
#         gex_only=False)
#     }

In [78]:
# Read each sequencing run data into a dictionary, with a name for each as the key
adatas = {
    "D1" : sc.read_10x_mtx(
        "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/collect_tube_1_batch_3_June_outs/filtered_feature_bc_matrix",
        var_names="gene_symbols",
        cache=False,
        make_unique=True,
        gex_only=False),
    "D2" : sc.read_10x_mtx(
        "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/Isol_Microglia_EFAD_TD_august_outs/filtered_feature_bc_matrix",
        var_names="gene_symbols",
        cache=False,
        make_unique=True,
        gex_only=False),
    "D3" : sc.read_10x_mtx(
        "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/Tube2_July_20_TD_outs/filtered_feature_bc_matrix",
        var_names="gene_symbols",
        cache=False,
        make_unique=True,
        gex_only=False),
    "D4" : sc.read_10x_mtx(
        "/Users/loganlarlham/Documents/Summer_proj_2024/Results_HT/Tube3_july_20_TD_outs/filtered_feature_bc_matrix",
        var_names="gene_symbols",
        cache=False,
        make_unique=True,
        gex_only=False),
    }

In [79]:
print(adatas['D1'])
print(adatas['D1'].var.head(20))
print(adatas['D1'].obs)
HTO1 = 0
for cell in adatas['D1'].obs.index:
    if cell[-1] == '1':
        HTO1 += 1
print(HTO1)
print(adatas['D1'].obs.index)


AnnData object with n_obs × n_vars = 3723 × 33993
    var: 'gene_ids', 'feature_types'
                         gene_ids    feature_types
Gm29155        ENSMUSG00000100764  Gene Expression
Gm29157        ENSMUSG00000100635  Gene Expression
Gm29156        ENSMUSG00000100480  Gene Expression
Pcmtd1         ENSMUSG00000051285  Gene Expression
Gm26901        ENSMUSG00000097797  Gene Expression
Gm30414        ENSMUSG00000103067  Gene Expression
Cdh7           ENSMUSG00000026312  Gene Expression
Exo1           ENSMUSG00000039748  Gene Expression
Becn2          ENSMUSG00000104158  Gene Expression
Uxs1           ENSMUSG00000057363  Gene Expression
Cdh19          ENSMUSG00000047216  Gene Expression
Gm29088        ENSMUSG00000101253  Gene Expression
Dsel           ENSMUSG00000038702  Gene Expression
Gm28189        ENSMUSG00000101453  Gene Expression
9330185C12Rik  ENSMUSG00000097648  Gene Expression
Pld5           ENSMUSG00000055214  Gene Expression
Rbm8a2         ENSMUSG00000078184  Gene Expres

In [86]:
for index, row in adatas['D1'].var.iterrows():
    if row['feature_types'] == 'Antibody Capture':
        print(row)
        




gene_ids                Hashtag_1
feature_types    Antibody Capture
Name: K231, dtype: object
gene_ids                Hashtag_2
feature_types    Antibody Capture
Name: K233, dtype: object
gene_ids                Hashtag_3
feature_types    Antibody Capture
Name: K234, dtype: object
gene_ids                Hashtag_4
feature_types    Antibody Capture
Name: K219, dtype: object


In [102]:
print(sum(adatas['D1'].var.gene_ids.str.startswith('Hashtag')))


4


In [103]:
hto=adatas['D1'][:,adatas['D1'].var.gene_ids.str.startswith('Hashtag')].copy()

In [104]:
hto

AnnData object with n_obs × n_vars = 3723 × 4
    var: 'gene_ids', 'feature_types'

In [107]:
hashsolo.hashsolo(hto)

In [109]:
hto.obs.head()

,most_likely_hypothesis,cluster_feature,negative_hypothesis_probability,singlet_hypothesis_probability,doublet_hypothesis_probability,Classification
AAACCCACACACCTAA-1,1.0,0.0,4.464767e-17,0.996940,0.003060,K219
AAACCCACACTGTGTA-1,1.0,0.0,6.267321e-17,0.999967,0.000033,K219
AAACCCAGTTCTTGCC-1,1.0,0.0,3.610621e-17,0.978346,0.021654,K231
AAACGAATCATTCCTA-1,1.0,0.0,4.542338e-17,0.997457,0.002543,K234
AAACGCTAGACGCCAA-1,1.0,0.0,1.346882e-10,0.999696,0.000304,K231


In [113]:
print(len(hto[hto.obs['most_likely_hypothesis']==1]))
print(len(hto[hto.obs['Classification']=='K219']))
print(len(hto[hto.obs['Classification']=='K231']))
print(len(hto[hto.obs['Classification']=='K234']))
print(len(hto[hto.obs['Classification']=='K233']))

View of AnnData object with n_obs × n_vars = 320 × 4
    obs: 'most_likely_hypothesis', 'cluster_feature', 'negative_hypothesis_probability', 'singlet_hypothesis_probability', 'doublet_hypothesis_probability', 'Classification'
    var: 'gene_ids', 'feature_types'
874
798
880
851


In [91]:
# Print Quality Control metric for each dataset
for adata_key, adata in adatas.items():
    print(f"{adata_key}:")
    summarize_adata(adata)
    print("\n")

D1:
Number of cells with fewer than 500 UMI counts: 0
Number of cells with fewer than 200 genes: 0
Number of genes expressed in fewer than 3 cells: 18651
Current Anndata has 3723 cells and 33993 genes, with a total amount of 34316840.0 UMI counts


D2:
Number of cells with fewer than 500 UMI counts: 0
Number of cells with fewer than 200 genes: 0
Number of genes expressed in fewer than 3 cells: 18124
Current Anndata has 5584 cells and 33993 genes, with a total amount of 38237432.0 UMI counts


D3:
Number of cells with fewer than 500 UMI counts: 0
Number of cells with fewer than 200 genes: 0
Number of genes expressed in fewer than 3 cells: 17804
Current Anndata has 4757 cells and 33993 genes, with a total amount of 40150808.0 UMI counts


D4:
Number of cells with fewer than 500 UMI counts: 0
Number of cells with fewer than 200 genes: 2
Number of genes expressed in fewer than 3 cells: 18694
Current Anndata has 4010 cells and 33993 genes, with a total amount of 34287456.0 UMI counts


